# PV181 Seminar 02- RNG (python)

This notebook contains python code for several tasks treated in this seminar. 

# PRNG 

# Determinism of PRNG

We will work with PRNG implemented in [random](https://docs.python.org/3/library/random.html) package. See first 4 methods (`seed, setstate, getstate, randbytes`) in the documentation. 
- **Task 1.** Import **random** package.
- **Task 2.** Generate (and print) 10 random bytes.  
- **Task 3.** Print out bytes in hexadecimal form (use `.hex()` method of bytes). Execute cell 2x. </br> PRNG produced different results since  generation is not deterministic as it is seeded by time. 
- **Task 4.** Use fixed `seed` and verify that generation is deterministic (generated bytes are always the same).     
- **Task 5.** Verify that seed determines internal state of the generator. What is the internal state for `seed = 1`? 
- **Task 6.** Use `seed = 1` and generate 10 bytes. Use the internal state from previous step 5. and generate the same 10 bytes - you should see same bytes.  
- **Task 7.** **Attack**: The generator produced 16 bytes that will be used as AES key. The first half of is `73a9bef499bbf4dca4f2`. Find the rest of the key. </br> **Hint**: user used small seed.

# LCG
Standard PRNG functions are very fast but also very weak. 
 * In python, PRNG [implemented](https://svn.python.org/projects/python/branches/release32-maint/Lib/random.py) in random module is [Mersenne Twister](https://en.wikipedia.org/wiki/Mersenne_Twister) with state formed by 625 32-bit integers. See size of the state in Task 1 above.
 * In other languages (C, Java, Rust) LCG is typically used. Internal state of LCG is **single** value (state) updated iterativelly as $$state = (state*a+c) \pmod m.$$ Overview of constants `a,c,m` used by the LCG for several languages can be found [here](https://en.wikipedia.org/wiki/Linear_congruential_generator). 
 </br>
 <span style="color:red">In LCG, state (new or old) is typically returned as generated random value!!</span>

# Standard PRNG  
Following code was taken from [ANSI C standard](http://www.open-std.org/jtc1/sc22/wg14/www/docs/n1256.pdf#page=324) and simplified to other portable implementation (according to implementation of [rand()](https://code.woboq.org/userspace/glibc/stdlib/random_r.c.html#__random_r)) of seeding function `srand` and function for generation `rand()`. 

```
static unsigned long int next = 1;

void srand(unsigned int seed)
{
    next = seed;
}

int rand(void) 
{
    return next = (next * 1103515245 + 12345) & 0x7fffffff;
}
```
- **Task 8.** Implement pythonic version of ANSI C PRNG - implement it as `class PRNG` below. </br> Use constants $a=1103515245, c=12345, m=2^{31}$ from [LCG wikipedia](https://en.wikipedia.org/wiki/Linear_congruential_generator).  
- **Task 9.** Generate 10 values $12345, 1406932606, ...$ with LCG seeded by 0.  
- **Task 10.** Generate 10 values but use different seed so the sequence will start with $1406932606, ...$.  

In [1]:
class PRNG:
    def __init__(self):
        pass #replace pass by few lines

    def srand(self, seed):
        pass #replace pass by few lines
    def rand(self):
        pass #replace pass by few lines

ansi_rand = PRNG()
# add seeding and generation of values

- **Task 11.** Every PRNG generates values in cycle i.e. generated sequence is periodic. Find the seed of the generator for which the generated sequence would be `[??, ??, 12345, 1406932606]`. Find previous two values (replaced by ??). </br>
 **Hint**:
 To revert this PRNG you can use constants $a^{-1}$ and $(c*a^{-1})$ instead of constants $a$ and $c$.  </br>
 Compute $a^{-1}$ and $(c*a^{-1})$ for ANSI C and generate (`[1406932606, 12345, ??, ??, seed]`).</br>
 In order to find backward LCG  it suffices to invert the update function: $$new\_state = old\_state*a+c \pmod m.$$   
 The inverse function can be computed as 
    $$
    \begin{align}
     old\_state &= (new\_state - c)/a \pmod m \\
                &= new\_state*(a^{-1}) - (c*a^{-1}) \pmod m \\
    \end{align}
    $$
  where $a^{-1} \pmod m$ can be computed using `*pow(a,-1,m)`. </br>

# Small state attack
- **Task 12.** Use rng and generate random bytes which will be used below as AES key (16B). Use `os.urandom` to generate seed (of one byte) of PRNG.

In [11]:
from cryptography.hazmat.primitives import hashes

def SHA1(message: bytes):
    digest = hashes.Hash(hashes.SHA1())
    digest.update(message)
    return digest.finalize() 

def SHA256(message: bytes):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(message)
    return digest.finalize() 

class PRNG:
    def __init__(self):
        self.srand(b'0x00')

    def srand(self, seed):
        if isinstance(seed, int):
            self.state = seed.to_bytes(1, 'little')
        else:
            self.state = seed
        self.state = self.state[:1]

    def rand_bytes(self, num_bytes=10):
        rnd = SHA1(self.state)[:num_bytes]
        self.state = SHA256(self.state)[:1]
        return bytes(rnd)

rng = PRNG()
seed = None # replace use os.urandom to generate 1 random byte as a seed
 # use seed to initalize the PRNG
K = None # generate the K

- **Task 13.** Use `encrypt_ECB` and `decrypt_ECB` to encrypt and decrypt arbitrary but short(16B) message.

In [12]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

def encrypt_ECB(key, msg):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    enc = cipher.encryptor()
    ct = enc.update(msg) + enc.finalize()
    return ct 

def decrypt_ECB(key, ct):
    cipher = Cipher(algorithms.AES(key),  modes.ECB())
    dec = cipher.decryptor()
    pt = dec.update(ct) + dec.finalize()
    return pt

K = b'' # replace b'' by 16B key generated by PRNG from the task 1.
ct = encrypt_ECB(K, b'') # replace b'' by message 
pt = decrypt_ECB(K, b'') # replace b''ciphertext ct
print(ct)
print(pt) # this should be your decrypted message

ValueError: Invalid key size (0) for AES.

- **Task 14.** **Attack**: Find all possible 16 byte blocks `rng` can produce and decrypt the ciphertext `ct`. The internal state is only 1 byte long and can be set to arbitrary value using ``.srand()`` method.     

# TRNG

# Sources: dev/random, dev/urandom
These two files provide secure way to generate random bytes! 
Reading from dev/urandom can by done using following functions: 

In [13]:
import os
import secrets 
os.urandom(10)
print(secrets.token_bytes(10).hex())
print(os.getrandom(10).hex())

d43b75e4412a6e4d902a
83f45bbbe90ad286f41f


Files **dev/random**, **dev/urandom** can be also opened as binary file for reading.  
Then you can read specified number of bytes e.g. 10. 

In [14]:
random_source = open("/dev/random", "rb")
random_source.read(10)

b"\xc9'\xdcE\x9a\xb5\x1a\xe4}1"

# Testing correlation of bits
- **Task 15.** Implement function `histogram(rnd_bytes, i, j)` that computes histogram of combination of bits (`i`-th and `j`-th bits of each byte). The function should return 4 frequencies for all bytes in `rnd_bytes` array. Frequencies will correspond to counts of how many bytes have `i`-th and `j`-th bit equal to combination 00,01,10 or 11.


In [18]:
import os
def histogram(rnd_bytes, i, j):
    hist = None
    return hist 

- **Task 16.**Verify that for arbitrary params `i,j` and size of generated block the frequencies are roughly equal.  

In [19]:
rnd_bytes = os.urandom(1000)
histogram(rnd_bytes, 0, 1)

- **Task 17.** Generate random bytes using ansi_rand = generate integers, apply modulo, transform to bytes (use `bytes()`).
- **Task 18.** Find params `i,j` where all the frequencies are exacly the same.  
 <span style="color:red"> Generator with such perfect results is also problematic! </span>   
 Can we predict some (next) bits with better probability than 50% ?

In [17]:
ansi_rand.srand(6)
rnd_bytes = bytes([])
histogram(rnd_bytes, 0, 1)